## Random forest regressor best params (this will continue 5 or 6 Random forest regression models from Initial modleing to see If I can get over .80 R2 score with random forest)

In [52]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

#### Read in

In [53]:
filepath = '../../processed_data/all_merged.csv'
all_enviro = pd.read_csv(filepath)

#### testing different GS parameters

In [54]:
X = all_enviro[['total population','ozone','pm2.5','diesel pm','pesticides','traffic','cleanup sites',
 'groundwater threats','haz. waste','imp. water bodies','solid waste','pollution burden',
'education','linguistic isolation','poverty','pop. char. ','drinking water',
 'tox. release','unemployment','ces_per','housing burden','est total',
 'est gen','est cold','est farm','est other']]
y = all_enviro['asthma']

###### clean before Splitting

In [55]:
X=pd.get_dummies(X)
X=X.fillna(X.median())

#get rid of missing target columns
X = X[~y.isna()]
y = y[~y.isna()]

In [56]:
X_train, X_test, y_train,y_test = train_test_split(X,y,random_state= 42)

###### Standard Scaler & fill with median

In [57]:
X_train=X_train.fillna(X_train.median())
X_test = X_test.fillna(X_test.median())

sc_X = StandardScaler()

X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [58]:
forest = RandomForestRegressor().fit(X_train,y_train)

In [59]:
print(f'Train R2 {forest.score(X_train,y_train)}')
print(f'Test R2 {forest.score(X_test,y_test)}')

Train R2 0.9629156043452666
Test R2 0.7317717245701832


#### Grid search different parameters

In [60]:
gsc = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid={ 
            "n_estimators"      : [10,100,150],
            "max_features"      : ["auto", "sqrt", "log2"],
            "max_depth" : [2,4,None],
            "bootstrap": [True, False],
            "min_samples_leaf": [1,3,5],
            },
            cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

In [61]:
forest_result = gsc.fit(X_train,y_train)

Fitting 3 folds for each of 162 candidates, totalling 486 fits


In [62]:
forest = RandomForestRegressor(**gsc.best_params_).fit(X_train, y_train)

In [63]:
best=gsc.best_params_
best

{'bootstrap': False,
 'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'n_estimators': 150}

In [64]:
print(f'Train accuracy {forest.score(X_train,y_train)}')
print(f'Test accuracy {forest.score(X_test,y_test)}')

Train accuracy 1.0
Test accuracy 0.7516377641754872
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, n_estimators=10; total time=   0.3s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=3, n_estimators=10; total time=   0.3s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=3, n_estimators=100; total time=   2.9s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=5, n_estimators=100; total time=   3.1s
[CV] END bootstrap=True, max_depth=2, max_features=sqrt, min_samples_leaf=3, n_estimators=100; total time=   0.7s
[CV] END bootstrap=True, max_depth=2, max_features=sqrt, min_samples_leaf=3, n_estimators=150; total time=   1.0s
[CV] END bootstrap=True, max_depth=2, max_features=log2, min_samples_leaf=1, n_estimators=150; total time=   1.0s
[CV] END bootstrap=True, max_depth=2, max_features=log2, min_samples_leaf=5, n_estimators=100; total time=   0.6s
[CV] END bootstrap=True, max_depth=4, 

- Hyperparameter tuning did not yield any noticible improvements to model score, suggesting that the default hyperparameters were close to optimal